# Apartment Price Prediction
## 1. Libraries

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import re

%matplotlib inline

## 2. Data Loading & Optimized Feature Engineering

In [ ]:
# Haversine Distance Function
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Layout Parsing
def parse_layout(layout):
    if pd.isna(layout): return 0, 0
    rooms = re.search(r'(\d+)', str(layout))
    rooms = int(rooms.group(1)) if rooms else 1
    kk = 1 if 'kk' in str(layout).lower() else 0
    return rooms, kk

# Condition Mapping
def map_condition(cond):
    mapping = {
        'Nový': 5,
        'Velmi dobrý': 4,
        'Dobrý': 3,
        'Udržovaný': 3,
        'Po rekonstrukci': 4,
        'Před rekonstrukcí': 2,
        'Špatný': 1,
        'Velmi špatný': 0,
        'Ve výstavbě': 5,
        'Projekt': 5
    }
    return mapping.get(cond, 3) # Default to average

# Load Data
train_df = pd.read_csv('appartments_train.csv')
test_df = pd.read_csv('appartments_test.csv')
print(len(test_df))

# Separate target and log-transform it
X = train_df.drop(columns=['price'])
y = train_df['price']
y_log = np.log1p(y)

X_test = test_df.copy()
if 'price' in X_test.columns:
    X_test = X_test.drop(columns=['price'])

# Combine
combined = pd.concat([X, X_test], axis=0).reset_index(drop=True)

print("Feature Engineering (Advanced)...Box")

# 1. Distance to Center
combined['dist_center'] = haversine_distance(combined['gps_lat'], combined['gps_lon'], 50.0812, 14.4280)
combined['dist_center_sq'] = combined['dist_center'] ** 2

# 2. Layout Parsing
combined[['n_rooms', 'has_kk']] = combined['layout'].apply(lambda x: pd.Series(parse_layout(x)))

# 3. Condition Refinement
combined['condition_num'] = combined['condition'].apply(map_condition)
combined['is_very_bad'] = (combined['condition'] == 'Velmi špatný').astype(int)
combined['is_new'] = (combined['condition'] == 'Nový').astype(int)

# 4. Date Features
combined['first_seen'] = pd.to_datetime(combined['first_seen'])
combined['last_seen'] = pd.to_datetime(combined['last_seen'])
min_date = combined['first_seen'].min()
combined['days_since_first_seen'] = (combined['first_seen'] - min_date).dt.days
combined['days_on_market'] = (combined['last_seen'] - combined['first_seen']).dt.days

# 5. Text Mining (Optimized: 500 Features, 30 SVD)
combined['text'] = combined['text'].fillna('').astype(str).str.lower()
keywords = {
    'luxus': r'luxus|nadstandard',
    'rekonstrukce': r'rekonstrukc|zrekonstru',
    'novostavba': r'novostavb|projekt',
    'metro': r'metro',
    'park': r'park',
    'balkon': r'balkon|lodži|terasa',
    'sklep': r'sklep|komora',
    'garaz': r'garáž|parkování|stání',
    'cihla': r'cihl'
}
for key, pattern in keywords.items():
    combined[f'has_{key}'] = combined['text'].str.contains(pattern, regex=True).astype(int)

tfidf = TfidfVectorizer(max_features=500, stop_words='english', ngram_range=(1, 2))
text_features = tfidf.fit_transform(combined['text'])
svd = TruncatedSVD(n_components=30, random_state=42)
text_pca = svd.fit_transform(text_features)
text_df = pd.DataFrame(text_pca, columns=[f'text_pca_{i}' for i in range(30)])
combined = pd.concat([combined, text_df], axis=1)

# 6. Geospatial Clustering (Optimized: 100 Clusters)
coords = combined[['gps_lat', 'gps_lon']].fillna(combined[['gps_lat', 'gps_lon']].mean())
kmeans = KMeans(n_clusters=100, random_state=42, n_init=10)
combined['loc_cluster'] = kmeans.fit_predict(coords)

# 7. Basic Cleaning
fill_zero_cols = ['cellar_area', 'balcony_area', 'garden_area', 'parking']
for col in fill_zero_cols:
    combined[col] = combined[col].fillna(0)
poi_nearest_cols = [c for c in combined.columns if 'nearest' in c]
for col in poi_nearest_cols:
    combined[col] = combined[col].fillna(combined[col].max() * 2.0)
combined['elevator'] = combined['elevator'].fillna('Unknown')

# 8. Ratios & Interactions
combined['floor_ratio'] = combined['floor'] / combined['total_floors']
combined['floor_ratio'] = combined['floor_ratio'].fillna(0)
combined['total_area'] = combined['area'] + combined['cellar_area'] + combined['balcony_area'] + combined['garden_area']

# Advanced Interactions
combined['cond_area'] = combined['condition_num'] * combined['total_area']
combined['cond_dist'] = combined['condition_num'] * combined['dist_center']
combined['renov_potential'] = combined['is_very_bad'] * combined['dist_center'] # Location value for dumps
combined['luxury_new'] = combined['is_new'] * combined['has_luxus']

# Drop columns
drop_cols = ['id', 'text', 'address', 'first_seen', 'last_seen']
combined = combined.drop(columns=drop_cols)

# Encode Categorical
cat_cols = ['layout', 'construction', 'condition', 'ownership', 'elevator']
combined = pd.get_dummies(combined, columns=cat_cols, drop_first=True)


X_train_raw = combined.iloc[:len(X)].copy()
X_test_raw = combined.iloc[len(X):].copy()

kf_te = KFold(n_splits=5, shuffle=True, random_state=42)
X_train_raw['cluster_target_enc'] = 0.0

for train_idx, val_idx in kf_te.split(X_train_raw, y_log):
    X_tr, X_val = X_train_raw.iloc[train_idx], X_train_raw.iloc[val_idx]
    y_tr = y_log.iloc[train_idx]
    
    # Compute means
    means = X_tr.groupby('loc_cluster').apply(lambda x: y_tr[x.index].mean())
    
    # Map to validation
    X_train_raw.loc[X_train_raw.index[val_idx], 'cluster_target_enc'] = X_val['loc_cluster'].map(means)

# Fill NaNs with global mean
global_mean = y_log.mean()
X_train_raw['cluster_target_enc'] = X_train_raw['cluster_target_enc'].fillna(global_mean)

# For Test: Map using full training data
full_means = X_train_raw.groupby('loc_cluster').apply(lambda x: y_log[x.index].mean())
X_test_raw['cluster_target_enc'] = X_test_raw['loc_cluster'].map(full_means).fillna(global_mean)


X_processed = X_train_raw.drop(columns=['loc_cluster'])
X_test_processed = X_test_raw.drop(columns=['loc_cluster'])

# Global Imputation
imputer = SimpleImputer(strategy='median')
X_processed_imputed = imputer.fit_transform(X_processed)
X_test_processed_imputed = imputer.transform(X_test_processed)
X_processed = pd.DataFrame(X_processed_imputed, columns=X_processed.columns)
X_test_processed = pd.DataFrame(X_test_processed_imputed, columns=X_test_processed.columns)

print(f"Processed shape: {X_processed.shape}")

1020
Feature Engineering (Advanced)...Box
Processed shape: (5000, 98)


/var/folders/pt/4rypzgx161xbyqfj39t1nrqh0000gn/T/ipykernel_44033/2597888005.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  means = X_tr.groupby('loc_cluster').apply(lambda x: y_tr[x.index].mean())
/var/folders/pt/4rypzgx161xbyqfj39t1nrqh0000gn/T/ipykernel_44033/2597888005.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  means = X_tr.groupby('loc_cluster').apply(lambda x: y_tr[x.index].mean())
/v

## 3. Stacking Ensemble with Optimized Hyperparameters

In [51]:
# Base Models (Optimized via Grid Search)
xgb_model = xgb.XGBRegressor(
    n_estimators=5000, 
    learning_rate=0.005, 
    max_depth=6, 
    subsample=0.8, 
    colsample_bytree=0.6, 
    reg_alpha=0.1, 
    reg_lambda=0.1,
    random_state=42, 
    n_jobs=-1
)

lgb_model = lgb.LGBMRegressor(
    n_estimators=3000, 
    learning_rate=0.005, 
    num_leaves=80, 
    subsample=0.8, 
    colsample_bytree=0.6, 
    reg_alpha=0.1, 
    reg_lambda=0.1,
    random_state=42, 
    n_jobs=-1, 
    verbose=-1
)

cb_model = cb.CatBoostRegressor(
    iterations=5000, 
    learning_rate=0.005, 
    depth=8, 
    l2_leaf_reg=3, 
    bagging_temperature=0.2,
    random_state=42, 
    verbose=False, 
    allow_writing_files=False
)

# Meta Learner
meta_learner = RidgeCV()

estimators = [
    ('xgb', xgb_model),
    ('lgb', lgb_model),
    ('cb', cb_model)
]

stacking_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_learner,
    cv=5,
    n_jobs=-1,
    passthrough=True
)

## 4. Final Prediction

In [53]:
# Refined Outlier Removal (Top 1% AND Bottom 0.5%)
upper_limit = y_log.quantile(0.99)
lower_limit = y_log.quantile(0.005)
mask = (y_log < upper_limit) & (y_log > lower_limit)

X_final_train = X_processed[mask]
y_final_train = y_log[mask]

print(f"Training on {len(X_final_train)} samples (removed {len(X_processed) - len(X_final_train)} outliers)...")

stacking_reg.fit(X_final_train, y_final_train)
final_preds_log = stacking_reg.predict(X_test_processed)
final_preds = np.expm1(final_preds_log)

# Bias Correction (Based on Rigorous Validation)
# Validation showed systematic under-prediction (Bias ~ 0.985)
correction_factor = 1.015
final_preds = final_preds * correction_factor
print(f"Applied bias correction factor: {correction_factor}")

submission = pd.DataFrame({
    'id': test_df['id'],
    'price': final_preds
})
submission.to_csv('Data_nerds_predikce.csv', index=False)
print("Submission saved to Data_nerds_predikce.csv")

Training on 4917 samples (removed 83 outliers)...
Applied bias correction factor: 1.015
Submission saved to Data_nerds_predikce.csv


In [54]:
submission.head()

,id,price
0,8795,7.307805e+06
1,6516,8.335489e+06
2,4714,5.178632e+06
3,8423,7.560859e+06
4,5361,7.543292e+06
